In [1]:
%pip install sentence-transformers
%pip install Qdrant-client

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from transformers import BartTokenizer
from langchain.text_splitter import TokenTextSplitter
import ollama
import shutil
import re
import json
from tqdm import tqdm
from langchain_community.document_loaders import PyPDFLoader

load_dotenv()

MODEL = "llama3.1:8b"

model = Ollama(model = MODEL)

C:\Users\User\AppData\Local\Temp\ipykernel_8704\597819308.py:17: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model = MODEL)


In [3]:

loader = PyPDFLoader("G:\\college stuff\\sem6\\NLP project indian\\Indian Penal Code Book.pdf")
pages = loader.load_and_split()
pages[0].page_content[:1000]


'THE INDIAN PENAL CODE\nCHAPTER I INTRODUCTION\nThe Indian Penal Code was drafted by the First Indian Law Commission presided over\nby Lord Thomas Babington Macaulay. The draft underwent further revision at the hands\nof well-known jurists, like Sir Barnes Peacock, and was completed in 1850. The Indian\nPenal Code was passed by the then Legislature on 6 October 1860 and was enacted as\nAct No. XLV of 1860.\nPreamble. WHEREAS it is expedient to provide a general\nPenal Code for India; It is enacted as follows:—\nCOMMENT.—The Indian Penal Code, 1860 (IPC, 1860) exhaustively codiﬁes the law\nrelating to offences with which it deals and the rules of the common law cannot be\nresorted to for inventing exemptions which are not expressly enacted.1. It is not\nnecessary and indeed not permissible to construe the IPC, 1860 at the present day in\naccordance with the notions of criminal jurisdiction prevailing at the time when the\nCode was enacted. The notions relating to this matter have very c

In [4]:
len(pages)

1963

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\User\AppData\Local\Temp\ipykernel_8704\1728591684.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [6]:
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# Start a Qdrant client (assumes local instance)
qdrant = QdrantClient(host="localhost", port=6333)

# Set collection name
collection_name = "indian_laws_collection"

# Check if collection exists
if not qdrant.collection_exists(collection_name=collection_name):
    # Only create if it doesn't exist
    qdrant.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=len(embedding_model.embed_query("test")), 
            distance=Distance.COSINE
        )
    )

# Create vector store (connect to the existing collection)
vectorstore = Qdrant(
    client=qdrant,
    collection_name=collection_name,
    embeddings=embedding_model,
)


C:\Users\User\AppData\Local\Temp\ipykernel_8704\1614656677.py:23: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(


In [7]:
vectorstore.add_documents(documents=pages)

['53672a7600df423ebe65656df8ec4aea',
 '2569651c41334c1c95599922787e77cb',
 '8bd02b04a82a4fb988680671209ec2ad',
 'e4edf62e328a41e09339eef38de4d681',
 'b9c9002c4f6648ba8b4bf6faab309ac6',
 '249f8c2af0af444aa45950435c284370',
 '71cb6a36841f4ef3bf39eedf161893ad',
 '2ca90dda199f401080cdbfdd0c83a155',
 '2991b85c78644cdab693bcd633c7116f',
 '7b99d9ae90b5481dab211fce23a5422e',
 'db43edcd63544af39b9ea59d136609e3',
 '3076d1da4b4f43a9a692f983b907033e',
 'f525fd0a6c414c8d916742e1da5ae4ac',
 'eeed4ada69de4b2a8a3ea4feebe0d3b7',
 '863bf155bd6f4e4b883c3347fd474b85',
 'b698d0c581f74b7d8af4ed7d17ee6608',
 '7d953d825b58405e8846c1df062cbe59',
 '0211438983184fadb1c2211ceae70150',
 '507c7b4aeedb486da88d0b033134e7e1',
 'ace6d774bdc246e2b21b16b5eea41c66',
 'c35a4c80b6f44158a820e4206d862242',
 '0748a0ac506b493cac5d4b9d574c7fa4',
 '3184544538b047d38ecbcf60fe9f6342',
 'a1bc9a22a43c458fae681e47adfa2ed6',
 '561dc7d2d06041bbad28d64bd528a8e4',
 '627e28571b2a436d9fd10d1a1a2b3531',
 '79027738db9f461e9ab133f5d0eded67',
 

In [8]:
#retriever can include google searches
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

def get_relevant_laws(data):
    results = retriever.invoke(data)
    return [doc.page_content for doc in results]



In [9]:
from langchain.prompts import PromptTemplate


template = """ 
Answer the questions based on the context below. If you cant answer the question, reply "I don't know".

Context: {context}

Question: {question}
 """

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="here is some context", question="here is some question"))

 
Answer the questions based on the context below. If you cant answer the question, reply "I don't know".

Context: here is some context

Question: here is some question
 


In [8]:
from langchain.schema.runnable import RunnableMap, RunnableLambda
from operator import itemgetter

# Define prompt
final_prompt = PromptTemplate(
    input_variables=["case_details", "retrieved_laws"],
    template="CASE DETAILS:\n{case_details}\n\nRELEVANT LAWS:\n{retrieved_laws}\n\nBased on the case details above, list the top 5 most relevant laws that would determine the outcome of this case. For each law, briefly explain why it applies. If you can't determine relevant laws, reply 'I don't know'."
)

# Define how to get context (e.g., via retriever)
# Assuming `retriever` is something like a vectorstore retriever
def retrieve_context(question):
    return {"context": retriever.get_relevant_documents(question)}

# Construct chain
chain = (
    RunnableMap({
        "retrieved_laws": lambda inputs: "\n\n".join([doc.page_content for doc in retriever.get_relevant_documents(inputs["question"])]),
        "case_details": itemgetter("question"),
    })
    | final_prompt
    | model
)

# Appending Relevant laws to case summaries

In [9]:

folder_path = "G:\\college stuff\\sem6\\NLP project indian\\fine-tune-data\\new"
output_folder = "G:\\college stuff\\sem6\\NLP project indian\\fine-tune-data\\dataset"
move_folder = "G:\\college stuff\\sem6\\NLP project indian\\fine-tune-data\\processed"

# Get total number of files for the progress bar
total_files = 0
for _, _, files in os.walk(folder_path):
    total_files += len(files)

# Create a progress bar
progress_bar = tqdm(total=total_files, desc="Processing Files", unit="file")

# List all files in the folder
file_count = 0
for root, dirs, files in os.walk(folder_path):
    
    for filename in files:
        
        file_path = os.path.join(root, filename)  # Create full file path
        
        with open(file_path, "r", encoding="utf-8") as f:  # Add encoding to avoid errors
            summary = f.read()
        
        # Invoke the chain and store the output
        result = chain.invoke({"question": summary})

        # Append the result to data2
        combined_data = summary + "\n\n<Relevant Laws>\n\n" + result + "\n\n</Relevant Laws>\n\n"

        # Write the combined data to a file
        with open(output_folder + "\\" + filename.replace(".txt", "+laws.txt"), "w", encoding="utf-8") as f:
            f.write(combined_data)
        
        # Move the processed file to the move_folder
        destination_path = os.path.join(move_folder, filename)
        shutil.move(file_path, destination_path)
        
        # Update progress bar
        progress_bar.update(1)
        progress_bar.set_postfix({"Current File": filename})
        
        file_count += 1
       

# Close the progress bar when done
progress_bar.close()
print(f"Completed processing {file_count} files.")


        

Processing Files:   0%|          | 0/28 [00:00<?, ?file/s]

C:\Users\User\AppData\Local\Temp\ipykernel_23132\265761928.py:18: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  "retrieved_laws": lambda inputs: "\n\n".join([doc.page_content for doc in retriever.get_relevant_documents(inputs["question"])]),
Processing Files: 100%|██████████| 28/28 [28:24<00:00, 60.86s/file, Current File=1665_pretrain.txt]

Completed processing 28 files.


# Converting case text + relevant laws to instruction, input, output format

In [ ]:

folder_path = "G:\\college stuff\\sem6\\NLP project indian\\fine-tune-data\\dataset"  #change this to your folder path
output_folder = "G:\\college stuff\\sem6\\NLP project indian\\fine-tune-data\\json"   #change this to your folder path
#move_folder = "G:\\college stuff\\sem6\\NLP project\\Case_summary+laws\\processed"

# Get total number of files for the progress bar
total_files = 0
for _, _, files in os.walk(folder_path):
    total_files += len(files)

# Create a progress bar
progress_bar = tqdm(total=total_files, desc="Processing Files", unit="file")


# List all files in the folder
file_count = 0
for root, dirs, files in os.walk(folder_path):
    
    for filename in files:
        
        file_path = os.path.join(root, filename)  # Create full file path
        
        with open(file_path, "r", encoding="utf-8") as f:  # Add encoding to avoid errors
            combined_data = f.read()

        facts_match = re.search(r'<FACTS>(.*?)(?=</FACTS>)', combined_data, re.DOTALL)
        arguments_match = re.search(r'<ARGUMENTS>(.*?)(?=</ARGUMENTS>)', combined_data, re.DOTALL)
        observations_match = re.search(r'<OBSERVATIONS>(.*?)(?=</OBSERVATIONS>)', combined_data, re.DOTALL)
        judgement_match = re.search(r'<JUDGMENT>(.*?)(?=</JUDGMENT>)', combined_data, re.DOTALL)
        laws_match = re.search(r'<Relevant Laws>(.*?)(?=</Relevant Laws>)', combined_data, re.DOTALL)

        # Extract the text or use empty string if not found
        facts = facts_match.group(1).strip() if facts_match else ""
        arguments = arguments_match.group(1).strip() if arguments_match else ""
        observations = observations_match.group(1).strip() if observations_match else ""
        judgement = judgement_match.group(1).strip() if judgement_match else ""
        laws = laws_match.group(1).strip() if laws_match else ""

        # Create the JSON object
        json_output = {
        "instruction": "You are a judge. Based on the following facts, legal arguments, court observations, and relevant laws, analyze the case thoroughly and predict a well-reasoned outcome and if you don't know the answer, reply 'I don't know'.",
        "input": f"Facts:\n{facts}\n\nArguments:\n{arguments}\n\nobservations:\n{observations}\n\nRelevant Laws:\n{laws}",
        "output": f"Decision Made: {judgement}"
        }

        # Write the JSON to a file
        with open(output_folder + "\\" + filename.replace(".txt", "_finetune.json"), "w", encoding="utf-8") as f:
            json.dump(json_output, f, indent=4)

        # # Move the processed file to the move_folder
        # destination_path = os.path.join(move_folder, filename)
        # shutil.move(file_path, destination_path)

        # Update progress bar
        progress_bar.update(1)
        progress_bar.set_postfix({"Current File": filename})
        
        file_count += 1
        

# Close the progress bar when done
progress_bar.close()
print(f"Completed processing {file_count} files.")

        

Processing Files: 100%|██████████| 503/503 [00:13<00:00, 38.57file/s, Current File=9_pretrain+laws.txt]   


Completed processing 503 files.


# Creating Dataset for Fine-tuning

In [4]:
import os
import json
import glob

def combine_json_files(input_directory, output_file):
    """
    Combine multiple JSON files in a directory into a single JSON file
    
    Args:
        input_directory: Directory containing JSON files to combine
        output_file: Path to the output combined JSON file
    """
    # List to store all JSON data
    combined_data = []
    
    # Find all JSON files in the directory
    json_files = glob.glob(os.path.join(input_directory, "*.json"))
    
    print(f"Found {len(json_files)} JSON files to combine")
    
    # Process each JSON file
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                combined_data.append(data)
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
    
    # Write combined data to output file
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(combined_data, outfile, indent=2)
        

In [5]:
input_dir = "G:\\college stuff\\sem6\\NLP project indian\\fine-tune-data\\json"  # Directory containing your JSON files
output_file = "finetune_data.json"  # Path for the combined output file
    
combine_json_files(input_dir, output_file)

Found 403 JSON files to combine
